In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.5 MB/s eta 0:00:00


In [ ]:
!pip install gdown

학습 데이터와 테스트 데이터 다운로드 방법은 e-book을 통해서만 제공합니다.



In [ ]:
import pandas as pd

In [ ]:
DATA_TRAIN_PATH = 'summ_train.json'
train_df = pd.read_json(DATA_TRAIN_PATH)
train_df = train_df.dropna()
train_df = train_df[:40000]
len(train_df)

40000

In [ ]:
DATA_TEST_PATH = 'summ_test.json'
test_df = pd.read_json(DATA_TEST_PATH)
test_df = test_df.dropna()
test_df = test_df[:5000]
len(test_df)

5000

In [ ]:
train_df.head()

,name,delivery_date,documents
0,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741778', 'category': '종합', 'media_t..."
1,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741792', 'category': '종합', 'media_t..."
2,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741793', 'category': '스포츠', 'media_..."
3,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741794', 'category': '정치', 'media_t..."
4,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741797', 'category': '종합', 'media_t..."


In [ ]:
test_df.head()

,name,delivery_date,documents
0,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626877', 'category': '정치', 'media_t..."
1,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626896', 'category': '종합', 'media_t..."
2,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626904', 'category': 'IT,과학', 'medi..."
3,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340627450', 'category': '사회', 'media_t..."
4,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340627465', 'category': '경제', 'media_t..."


In [ ]:
def preprocess_data(data):
    outs = []
    for doc in data['documents']:
        line = []
        line.append(doc['media_name'])
        line.append(doc['id'])
        para = []
        for sent in doc['text']:
            for s in sent:
                para.append(s['sentence'])
        line.append(para)
        line.append(doc['abstractive'][0])
        line.append(doc['extractive'])
        a = doc['extractive']
        if a[0] == None or a[1] == None or a[2] == None:
            continue
        outs.append(line)

    outs_df = pd.DataFrame(outs)
    outs_df.columns = ['media', 'id', 'article_original', 'abstractive', 'extractive']
    return outs_df

In [ ]:
# 원문과 요약문을 각각 'article_original'와 'abstractive'열에 저장.
train_data = preprocess_data(train_df)
train_data.head()

,media,id,article_original,abstractive,extractive
0,광양신문,290741778,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,"[2, 3, 10]"
1,광양신문,290741792,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,"[2, 4, 11]"
2,광양신문,290741793,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,"[3, 5, 7]"
3,광양신문,290741794,"[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...","[2, 3, 4]"
4,광양신문,290741797,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,"[3, 7, 4]"


In [ ]:
# 원문과 요약문을 각각 'article_original'와 'abstractive'열에 저장.
test_data = preprocess_data(test_df)
test_data.head()

,media,id,article_original,abstractive,extractive
0,한국경제,340626877,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"
1,한국경제,340626896,"[] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다., 바른미래당이 8일 ...",8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,"[2, 1, 6]"
2,한국경제,340626904,"[[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀., 지난 3일 한국이 세계 최초...",지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,"[1, 5, 8]"
3,한국경제,340627450,[] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,"[0, 1, 2]"
4,한국경제,340627465,"[[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,"[1, 3, 4]"


In [ ]:
# train_data의 첫번째 샘플의 article_original 열의 값 출력
train_data['article_original'].loc[0]

['ha당 조사료 400만원…작물별 차등 지원',
 '이성훈 sinawi@hanmail.net',
 '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.',
 '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.',
 '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.',
 '지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다.',
 '지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.',
 '농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다.',
 'ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다.',
 '벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다.',
 '논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다.',
 '전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다.',
 '최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.']

In [ ]:
train_data['news'] = train_data['article_original'].apply(lambda x: ' '.join(x))
test_data['news'] = test_data['article_original'].apply(lambda x: ' '.join(x))

In [ ]:
# train_data의 첫번째 샘플의 news 열의 값 출력
train_data['news'].loc[0]

'ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.'

In [ ]:
train_data[['news', 'abstractive']].head()

,news,abstractive
0,ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


In [ ]:
import logging
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
from tqdm import tqdm

In [ ]:
class KoBARTSummaryDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, ignore_index=-100):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_len = max_len
        # self.docs = pd.read_csv(file, sep='\t')
        self.docs = df
        self.len = self.docs.shape[0]

        self.pad_index = self.tokenizer.pad_token_id
        self.ignore_index = ignore_index

    def add_padding_data(self, inputs):
        # 인코더의 입력과 디코더의 입력은 0으로 패딩
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs

    def add_ignored_data(self, inputs):
        # 디코더의 레이블은 -100으로 패딩
        if len(inputs) < self.max_len:
            pad = np.array([self.ignore_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs

    def __getitem__(self, idx):
        instance = self.docs.iloc[idx]
        input_ids = self.tokenizer.encode(instance['news'])
        input_ids = self.add_padding_data(input_ids)

        label_ids = self.tokenizer.encode(instance['abstractive'])
        label_ids.append(self.tokenizer.eos_token_id)
        dec_input_ids = [self.tokenizer.eos_token_id]
        dec_input_ids += label_ids[:-1]
        dec_input_ids = self.add_padding_data(dec_input_ids)
        label_ids = self.add_ignored_data(label_ids)

        return {'input_ids': np.array(input_ids, dtype=np.int_),
                'decoder_input_ids': np.array(dec_input_ids, dtype=np.int_),
                'labels': np.array(label_ids, dtype=np.int_)}

    def __len__(self):
        return self.len

In [ ]:
class KoBARTConditionalGeneration(torch.nn.Module):
    def __init__(self):
        super(KoBARTConditionalGeneration, self).__init__()
        self.model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v1')
        self.tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')
        self.pad_token_id = self.tokenizer.pad_token_id

    def forward(self, inputs):
        attention_mask = inputs['input_ids'].ne(self.pad_token_id).float()
        decoder_attention_mask = inputs['decoder_input_ids'].ne(self.pad_token_id).float()

        return self.model(input_ids=inputs['input_ids'],
                          attention_mask=attention_mask,
                          decoder_input_ids=inputs['decoder_input_ids'],
                          decoder_attention_mask=decoder_attention_mask,
                          labels=inputs['labels'], return_dict=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = KoBARTConditionalGeneration().to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
batch_size = 32
max_len = 512
num_workers = 4
lr = 3e-5
max_epochs = 10
warmup_ratio = 0.1

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')
train_dataset = KoBARTSummaryDataset(train_data, tokenizer, max_len=max_len)
test_dataset = KoBARTSummaryDataset(test_data, tokenizer, max_len=max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

optimizer = AdamW(model.parameters(), lr=lr)
total_steps = len(train_loader) * max_epochs
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=int(total_steps * warmup_ratio), T_mult=1, eta_min=0)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
print('첫번째 샘플의 원문 텍스트 :', train_data['news'].loc[0])

첫번째 샘플의 원본 텍스트 : ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.


In [ ]:
print('첫번째 샘플의 원문 텍스트의 정수 인코딩 및 패딩 결과 :', train_dataset[0]['input_ids'])
print('정수 인코딩 및 패딩 후의 길이 :', len(train_dataset[0]['input_ids']))

첫번째 샘플의 원문 텍스트의 정수 인코딩 및 패딩 결과 : [21582   296  9770 14666 10386 14136 16266 14476 12061 10675 10872 14165
 10001 14423 19168 13758 18482 14885   296   318   304   263   303 15195
   308   296 17761   245 26818 26102  9506 14973 18193 24873 24777 27841
 25486 14185 26379 15358 14049 18193 14871 17074 14902 15055 14634 17613
 19754 18193 14871 17074 16245 19609 10443 14291 15912 14396 11786 19609
 15410 14031 10386 12061 10675 14303 20054 14048 14355 14212 15049 14291
 14553 19609 15231 16864 16365 15828 26030 19560 16617 14130 14516 16309
 12024 14396 14355 14212 10675 19696 14491 20087 12005 14770 21598 26407
 12024 14383 16167   236 12037 16248 14195 25292 26407 15888 20066 29246
 12332 21405 16053   243 14044 20169 19304 15615 14423 14573 24884 19961
 11211 18193 19819 12333 10955 14842 15141 14497 15261 14239   296 15377
 14079 13455   240 14333 19609 17901 14355 14212 15049 19696 13590 18102
 14635 14130 14423 14576 14212 19839 15623 17287 17400 14056  9584 20466
 14537 14212 15689

In [ ]:
print('첫번째 샘플의 정수 인코딩 후 복원 결과 :')
tokenizer.decode(train_dataset[0]['input_ids'])

첫번째 샘플의 정수 인코딩 후 복원 결과 :


'ha당 조사료 400만원...작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [ ]:
print('첫번째 샘플의 요약문:', train_data['abstractive'].loc[0])

첫번째 샘플의 요약문: 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .


In [ ]:
print('첫번째 샘플의 요약문 텍스트의 정수 인코딩 및 패딩 결과', train_dataset[0]['decoder_input_ids'])
print('정수 인코딩 및 패딩 후의 길이 :', len(train_dataset[0]['decoder_input_ids']))

첫번째 샘플의 요약문 텍스트의 정수 인코딩 및 패딩 결과 [    1 26102  9506 14973 18193 24873 24777 27841 25486 14185 26379 19609
 10443 14291 15912 14396 11786 19609 15410 14031 10386 12061 10675 14303
 20054 14048 14355 14212 15049 14291 14553 19609 15231 16864 16365 15828
 26030 19560 14063 11495 14871 17074 12147 15127 17489 15358 15272 14432
 14834 15271 15243 15869 15450 15364 14497 12332 16516 12332 23891 10586
  9879 17982 18290 15453 17210  9754 17546     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3 

In [ ]:
print('첫번째 샘플의 정수 인코딩 및 패딩 후 복원 결과 :')
tokenizer.decode(train_dataset[0]['decoder_input_ids'])

첫번째 샘플의 정수 인코딩 및 패딩 후 복원 결과 :


"</s> 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [ ]:
# -100의 값은 tokenizer_decode하면 에러나므로 임시로 3으로 변경 후 출력
test_array = train_dataset[0]['labels']
test_array[test_array == -100] = 3
print('첫번째 샘플의 요약문 레이블 :', tokenizer.decode(test_array))

첫번째 샘플의 요약문 레이블 : 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [ ]:
best_loss = np.inf
for epoch in range(max_epochs):
    print(epoch+1, '수행 중')
    model.train()
    for batch in tqdm(train_loader, total=len(train_loader)):
        batch = {k: v.to(device) for k, v in batch.items()} # Move the batch tensors to the same device as the model
        optimizer.zero_grad()
        outputs = model(batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(test_loader, total=len(test_loader)):
            batch = {k: v.to(device) for k, v in batch.items()} # Move the batch tensors to the same device as the model
            outputs = model(batch)
            total_loss += outputs.loss.item()

    avg_loss = total_loss / len(test_loader)
    print(f'Epoch: {epoch+1}, Loss: {avg_loss}')

    # Save the best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), 'best_model.pt')
        print(f'Validation loss improved from {best_loss:.4f} to {avg_loss:.4f}. 체크포인트를 저장합니다.')

1 수행 중


  0%|          | 0/1250 [00:03<?, ?it/s]


OutOfMemoryError: ignored

In [ ]:
'''
a100 gpu로 배치 크기 45로 수행하였을 때

1 진행 중
100%|██████████| 889/889 [07:36<00:00,  1.95it/s]
100%|██████████| 112/112 [00:25<00:00,  4.37it/s]
Epoch: 1, Loss: 1.5542645347969872
2 진행 중
100%|██████████| 889/889 [07:27<00:00,  1.99it/s]
100%|██████████| 112/112 [00:25<00:00,  4.43it/s]
Epoch: 2, Loss: 1.4970393446939332
3 진행 중
 93%|█████████▎| 831/889 [06:58<00:29,  2.00it/s]
Epoch: 3, Loss: 1.4749100123132979
4 진행 중
100%|██████████| 889/889 [07:35<00:00,  1.95it/s]
100%|██████████| 112/112 [00:25<00:00,  4.44it/s]
Epoch: 4, Loss: 1.4651334785989352
5 진행 중
100%|██████████| 889/889 [07:37<00:00,  1.94it/s]
100%|██████████| 112/112 [00:25<00:00,  4.37it/s]
Epoch: 5, Loss: 1.4626388241137778
6 진행 중
100%|██████████| 889/889 [07:35<00:00,  1.95it/s]
100%|██████████| 112/112 [00:25<00:00,  4.43it/s]
Epoch: 6, Loss: 1.4644672806773866
7 진행 중
100%|██████████| 889/889 [07:36<00:00,  1.95it/s]
100%|██████████| 112/112 [00:25<00:00,  4.44it/s]
Epoch: 7, Loss: 1.4712920082466943
8 진행 중
  4%|▍         | 36/889 [00:19<07:49,  1.82it/s]
---------------------------------------------------------------------------
KeyboardInterrupt                         Traceback (most recent call last)
/tmp/ipykernel_75844/3256950047.py in
     53
     54 if __name__ == '__main__':
---> 55     train()

/tmp/ipykernel_75844/3256950047.py in train()
     32             outputs = model(batch)
     33             loss = outputs.loss
---> 34             loss.backward()
     35             optimizer.step()
     36             scheduler.step()

/opt/conda/lib/python3.8/site-packages/torch/_tensor.py in backward(self, gradient, retain_graph, create_graph, inputs)
    305                 create_graph=create_graph,
    306                 inputs=inputs)
--> 307         torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
    308
    309     def register_hook(self, hook):

/opt/conda/lib/python3.8/site-packages/torch/autograd/__init__.py in backward(tensors, grad_tensors, retain_graph, create_graph, grad_variables, inputs)
    152         retain_graph = create_graph
    153
--> 154     Variable._execution_engine.run_backward(
    155         tensors, grad_tensors_, retain_graph, create_graph, inputs,
    156         allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag

KeyboardInterrupt:
'''

"\na100 gpu로 배치 크기 45로 수행하였을 때\n\n1 진행 중\n100%|██████████| 889/889 [07:36<00:00,  1.95it/s]\n100%|██████████| 112/112 [00:25<00:00,  4.37it/s]\nEpoch: 1, Loss: 1.5542645347969872\n2 진행 중\n100%|██████████| 889/889 [07:27<00:00,  1.99it/s]\n100%|██████████| 112/112 [00:25<00:00,  4.43it/s]\nEpoch: 2, Loss: 1.4970393446939332\n3 진행 중\n 93%|█████████▎| 831/889 [06:58<00:29,  2.00it/s]\nEpoch: 3, Loss: 1.4749100123132979\n4 진행 중\n100%|██████████| 889/889 [07:35<00:00,  1.95it/s]\n100%|██████████| 112/112 [00:25<00:00,  4.44it/s]\nEpoch: 4, Loss: 1.4651334785989352\n5 진행 중\n100%|██████████| 889/889 [07:37<00:00,  1.94it/s]\n100%|██████████| 112/112 [00:25<00:00,  4.37it/s]\nEpoch: 5, Loss: 1.4626388241137778\n6 진행 중\n100%|██████████| 889/889 [07:35<00:00,  1.95it/s]\n100%|██████████| 112/112 [00:25<00:00,  4.43it/s]\nEpoch: 6, Loss: 1.4644672806773866\n7 진행 중\n100%|██████████| 889/889 [07:36<00:00,  1.95it/s]\n100%|██████████| 112/112 [00:25<00:00,  4.44it/s]\nEpoch: 7, Loss: 1.4712920082466

In [ ]:
'''
# 모델 인스턴스 생성
model_wrapper = KoBARTConditionalGeneration().to(device)

# 가중치 로드
model_wrapper.load_state_dict(torch.load('best_model.pt'))

# 모델을 평가 모드로 설정
model_wrapper.eval()
'''

"\n# 모델 인스턴스 생성\nmodel_wrapper = KoBARTConditionalGeneration().to(device)\n\n# 가중치 로드\nmodel_wrapper.load_state_dict(torch.load('best_model.pt'))\n\n# 모델을 평가 모드로 설정\nmodel_wrapper.eval()\n"

In [ ]:
text = test_data.loc[25]['news']
text

"배우 배수지가 매니지먼트 숲과 전속계약을 체결했다. 수지는 8일 자신의 인스타그램에 '데뷔 때 부터 함께해온 소속사 JYP와 계약기간을 마치고 오늘부터 새로운 소속사 매니지먼트 숲과 함께 하게 되었다'고 밝혔다. 이어 수지는 '연습생으로 시작해서, 데뷔하고 9년의 시간이 흐른 지금까지, JYP와 함께했던 여러 영광의 순간들이 스쳐지나간다'면서 '9년 동안 항상 옆에서 서포트 해주셨던 JYP 모든 직원분들께 진심으로 감사드린다'고 인사를 잊지 않았다. 2010년 걸그룹 '미쓰에이'로 데뷔한 배수지는 2011년 KBS2 드라마 '드림하이'로 첫 연기 활동을 시작했다. 2012년 영화 '건축학개론'을 통해 스크린 데뷔를 한 뒤 가수 활동과 연기 활동을 꾸준히 병행해 오고 있다. 매니지먼트 숲 관계자는 '배우 배수지의 장점과 매력을 극대화할 수 있는 작품 선택부터 국내외 활동, 가수로서의 솔로 활동까지 활발하게 이루어질 수 있도록 지원할 예정이다'고 전했다. 특히 올해는 작품을 통해 연기자 배수지로 대중들과 만날 예정이다. 현재 촬영 중인 SBS 드라마 '배가본드'는 민항 여객기 추락 사고에 연루된 한 남자가 은폐된 진실 속에서 찾아낸 거대한 국가 비리를 파헤치게 되는 과정을 담은 이야기다. 배수지는 국정원 블랙요원 고해리 역으로 출연하며, 뒤이어 영화 '백두산'에도 합류한다. 매니지먼트 숲은 공유, 공효진, 김재욱, 서현진, 이천희, 전도연, 정유미, 남지현, 최우식, 유민규, 이재준, 정가람, 전소니 등 소속되어 있다."

In [ ]:
label = test_data.loc[25]['abstractive']
label

'지난 8일 배수지는 자신의 인스타그램에 매니지먼트 숲과의 전속계약을 알리며 지난 9년 간 몸담았던 JYP 직원들에게 감사의 인사를 전했다.'

In [ ]:
'''
input_ids = tokenizer.encode(text)
input_ids = torch.tensor(input_ids)
input_ids = input_ids.unsqueeze(0).to(device)
output = model_wrapper.model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=5)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)
'''

In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 40.8 MB/s eta 0:00:00


In [ ]:
!pip install rouge

In [ ]:
from konlpy.tag import Okt
from rouge import Rouge

def calculate_rouge_for_korean_sentences(reference_sentence, hypothesis_sentence):
    # 형태소 분석기 초기화
    okt = Okt()

    # 형태소 분석을 통한 토큰화
    def tokenize_and_concat(text):
        return ' '.join(okt.morphs(text))

    # 두 문장 토큰화
    tokenized_reference = tokenize_and_concat(reference_sentence)
    tokenized_hypothesis = tokenize_and_concat(hypothesis_sentence)

    # ROUGE 점수 계산
    rouge = Rouge()
    scores = rouge.get_scores(tokenized_hypothesis, tokenized_reference)
    return scores

# 사용 예시
label = "고양이가 매트 위에 앉아 있다."
model1_prediction = "매트 위에 고양이가 앉아 있다."
model2_prediction = "고양이가 매트 위에 앉아 있다."

rouge_scores = calculate_rouge_for_korean_sentences(label, model1_prediction)
print(rouge_scores[0]['rouge-l']['f'])
rouge_scores = calculate_rouge_for_korean_sentences(label, model2_prediction)
print(rouge_scores[0]['rouge-l']['f'])

0.7142857092857143
0.999999995


In [ ]:
output = '배우 배수지가 매니지먼트 숲과 전속계약을 체결해 배우 배수지의 장점과 매력을 극대화할 수 있는 작품 선택부터 국내외 활동, 가수로서의 솔로 활동까지 활발하게 이루어질 수 있도록 지원할 예정이다.'
label = test_data.loc[25]['abstractive']
rouge_scores = calculate_rouge_for_korean_sentences(label, output)

print('모델의 예측:', output)
print('정답 문장:', label)
print('rouge score:', rouge_scores[0]['rouge-l']['f'])

모델의 예측: 배우 배수지가 매니지먼트 숲과 전속계약을 체결해 배우 배수지의 장점과 매력을 극대화할 수 있는 작품 선택부터 국내외 활동, 가수로서의 솔로 활동까지 활발하게 이루어질 수 있도록 지원할 예정이다.
정답 문장: 지난 8일 배수지는 자신의 인스타그램에 매니지먼트 숲과의 전속계약을 알리며 지난 9년 간 몸담았던 JYP 직원들에게 감사의 인사를 전했다.
rouge score: 0.16129031762747154
